In [ ]:
!pip install geopandas pandas pyarrow
import geopandas as gpd

In [ ]:
import pandas as pd
import shapely
import plotly.express as px

In [ ]:
roads=gpd.read_file('USA_Tennessee.geojson')
roads=roads[roads.County=='DAVIDSON']

In [ ]:
incidents=pd.read_parquet('nfd_incidents_xd_seg.parquet')
incidents['geometry']=incidents.geometry.apply(shapely.wkt.loads)
incidents = gpd.GeoDataFrame(incidents,geometry=incidents.geometry)

In [ ]:
weather=pd.read_parquet('weatherbit_weather_2010_2022.parquet')
distinctstations=weather[['station_id','gps_coordinate_latitude', 'gps_coordinate_longitude']].drop_duplicates().dropna()

In [ ]:
fig = px.scatter_mapbox(distinctstations, lat="gps_coordinate_latitude", hover_name='station_id', lon="gps_coordinate_longitude", size_max=15, zoom=10,mapbox_style='open-street-map')
